In [1]:
from core import *

In [2]:
bank1 = Bank()
bank2 = Bank()

In [3]:
# Поэтапное создание клиентов
client1 = Client(bank1, "Иван", "Иванов")
client1.address = "Москва"
client1.passport = "1234 567890"

client1Facade = ClientFacade(client1)
debit_1 = client1Facade.create_account(DebitAccount)

client2 = Client(bank2, "Петр", "Петров")
client2Facade = ClientFacade(client2)
debit_2 = client2Facade.create_account(DebitAccount)

In [4]:
# Внесение наличных

client1Facade.deposit(debit_1, 1000)
debit_1.balance

1000

In [5]:
# Перевод

Transaction(debit_1, debit_2, 500).perform()
debit_1.balance, debit_2.balance

(500, 500)

In [6]:
# Альтернативно, клиенты могут через фасад переводить по id счёта получателя

debit_2.id

UUID('4797619b-f65f-4595-87ff-5ca4df935cd9')

In [7]:
client1Facade.transfer(debit_1, debit_2.id, 500, bank2)
debit_1.balance, debit_2.balance

(0, 1000)

In [8]:
# Если просто попросить Петрова вернуть деньги, то он не сможет, т.к. у него нет паспорта

Transaction(debit_2, debit_1, 500).perform()

Error: Client doesn't have passport or address

In [9]:
# Но поможет механизм отмены транзакции
transactions = debit_1.history.see()
transactions

[*5e87 -> *7c1a: 1000 at 2023-03-15 20:35:32.870406,
 *35cd -> *7c1a: -500 at 2023-03-15 20:35:32.896032,
 *35cd -> *7c1a: -500 at 2023-03-15 20:35:32.964331]

In [10]:
transactions[-1].cancel()
transactions[-2].cancel()
debit_1.balance, debit_2.balance

(1000, 0)

In [11]:
# История с отрицательными транзакциями выглядит немного странно,
# но идея в том, что сумма всех чисел равна конечному балансу
debit_1.history.see()

[*5e87 -> *7c1a: 1000 at 2023-03-15 20:35:32.870406,
 *35cd -> *7c1a: -500 at 2023-03-15 20:35:32.896032,
 *35cd -> *7c1a: -500 at 2023-03-15 20:35:32.964331,
 *35cd -> *7c1a: 500 at 2023-03-15 20:35:33.031689,
 *35cd -> *7c1a: 500 at 2023-03-15 20:35:33.031730]